In [1]:
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import pymongo
import os

In [2]:
load_dotenv()

True

In [2]:
client = pymongo.MongoClient(os.getenv('MONGO_URI'))
db = client[os.getenv('MONGO_DATABASE')]
collection = db['scrapy_bogota_apartments']

df = pd.DataFrame(list(collection.find()))
df = df.drop(columns=['_id'], axis=1)

df.head(3)

,codigo,tipo_propiedad,tipo_operacion,precio_venta,precio_arriendo,area,habitaciones,banos,administracion,parqueaderos,...,featured_sector,descripcion,datetime,website,compañia,imagenes,fecha_actualizacion_precio_venta,precio_venta_anterior,fecha_actualizacion_precio_arriendo,precio_arriendo_anterior
0,374-M2718950,APARTAMENTO,VENTA Y ARRIENDO,2.500000e+09,13500000.0,320.0,4.0,5.0,2500000.0,3.0,...,"[CERCA CENTROS COMERCIALES, CERCA COLEGIOS / U...","vendo o arriendo ph duplex montearroyo, 320 mt...",2023-06-19 05:43:49.763,metrocuadrado.com,LIDIS MACIAS,[https://multimedia.metrocuadrado.com/374-M271...,NaT,NaN,NaT,NaN
1,2510-M4156145,APARTAMENTO,VENTA,4.850000e+08,NaN,84.0,3.0,3.0,680000.0,1.0,...,NaN,"oportunidad, bonito apartamento, recien refacc...",2023-06-19 05:43:53.411,metrocuadrado.com,JM GESTIONES INMOBILIARIAS,[https://multimedia.metrocuadrado.com/2510-M41...,NaT,NaN,NaT,NaN
2,10312-M2873283,APARTAMENTO,VENTA Y ARRIENDO,1.980000e+09,9000000.0,248.0,3.0,3.0,2200000.0,3.0,...,"[CERCA CENTROS COMERCIALES, CERCA COLEGIOS / U...",presentamos espectacular apartamento en uno de...,2023-06-19 05:43:58.765,metrocuadrado.com,PENTAGOLD,[https://multimedia.metrocuadrado.com/10312-M2...,NaT,NaN,NaT,NaN


In [4]:
df.columns

Index(['codigo', 'tipo_propiedad', 'tipo_operacion', 'precio_venta',
       'precio_arriendo', 'area', 'habitaciones', 'banos', 'administracion',
       'parqueaderos', 'sector', 'estrato', 'antiguedad', 'estado', 'longitud',
       'latitud', 'featured_interior', 'featured_exterior',
       'featured_zona_comun', 'featured_sector', 'descripcion', 'datetime',
       'website', 'compañia', 'imagenes', 'fecha_actualizacion_precio_venta',
       'precio_venta_anterior', 'fecha_actualizacion_precio_arriendo',
       'precio_arriendo_anterior'],
      dtype='object')

In [5]:
images_explode = df.explode('imagenes')
images_explode = images_explode.dropna(subset=['imagenes'])

In [6]:
images_df = images_explode[['codigo', 'imagenes']].rename(columns={'imagenes': 'url_imagen'})
images_df.shape

(343741, 2)

In [7]:
images_df.to_csv('../data/processed/images.csv', index=False)
images_df.columns

Index(['codigo', 'url_imagen'], dtype='object')

In [17]:
df = df.drop(columns=['imagenes'], axis=1)

In [8]:
def check_jacuzzi(x):
    if type(x) == list:
        return 1 if 'JACUZZI' in x else 0
    else:
        return 0

df['jacuzzi'] = df.featured_interior.apply(check_jacuzzi)

In [9]:
def extract_piso(x):
    if type(x) == list:
        try:
            for item in x:
                if item.startswith('PISO '):
                    return int(item.split(' ')[1])
        except:
            return np.nan
    else:
        return np.nan
    
    
df['piso'] = df.featured_interior.apply(extract_piso)

In [10]:
def extract_closets(x):
    if type(x) == list:
        try:
            for item in x:
                if item.startswith('CLOSETS'):
                    return int(item.split(' ')[1])
        except:
            return np.nan
    else:
        return np.nan
    
df['closets'] = df.featured_interior.apply(extract_closets)

In [11]:
def check_chimeny(x):
    if type(x) == list:
        return 1 if 'CHIMENEA' in x else 0
    else:
        return 0
    
df['chimenea'] = df.featured_interior.apply(check_chimeny)

In [12]:
def check_mascotas(x):
    if type(x) == list:
        return 1 if 'PERMITE MASCOTAS' in x else 0
    else:
        return 0
    
df['permite_mascotas'] = df.featured_interior.apply(check_mascotas)

In [13]:
def check_gimnasio(x):
    if type(x) == list:
        return 1 if 'GIMNASIO' in x else 0
    else:
        return 0
    
df['gimnasio'] = df.featured_zona_comun.apply(check_gimnasio)

In [14]:
def check_ascensor(x):
    if type(x) == list:
        return 1 if 'ASCENSOR' in x else 0
    else:
        return 0
    
df['ascensor'] = df.featured_exterior.apply(check_ascensor)

In [15]:
def check_conjunto_cerrado(x):
    if type(x) == list:
        return 1 if 'CONJUNTO CERRADO' in x else 0
    else:
        return 0
    
df['conjunto_cerrado'] = df.featured_exterior.apply(check_conjunto_cerrado)

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40177 entries, 0 to 40176
Data columns (total 36 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   codigo                               40177 non-null  object        
 1   tipo_propiedad                       40177 non-null  object        
 2   tipo_operacion                       40177 non-null  object        
 3   precio_venta                         27050 non-null  float64       
 4   precio_arriendo                      13895 non-null  float64       
 5   area                                 40175 non-null  float64       
 6   habitaciones                         40169 non-null  float64       
 7   banos                                40171 non-null  float64       
 8   administracion                       32021 non-null  float64       
 9   parqueaderos                         40169 non-null  float64       
 10  sector    

In [19]:
df = df.drop(columns=['featured_interior', 'featured_zona_comun', 'featured_exterior', 'featured_sector'], axis=1)
df.columns

Index(['codigo', 'tipo_propiedad', 'tipo_operacion', 'precio_venta',
       'precio_arriendo', 'area', 'habitaciones', 'banos', 'administracion',
       'parqueaderos', 'sector', 'estrato', 'antiguedad', 'estado', 'longitud',
       'latitud', 'descripcion', 'datetime', 'website', 'compañia',
       'fecha_actualizacion_precio_venta', 'precio_venta_anterior',
       'fecha_actualizacion_precio_arriendo', 'precio_arriendo_anterior',
       'jacuzzi', 'piso', 'closets', 'chimenea', 'permite_mascotas',
       'gimnasio', 'ascensor', 'conjunto_cerrado'],
      dtype='object')

In [20]:
df.to_csv('../data/interim/apartments.csv', index=False)

In [21]:
import session_info

session_info.show()